# ML Pipeline Preparation

 Follow the instructions below to help you create your ML pipeline.

### 1. Import libraries and load data from database.

Import Python libraries

Load dataset from database with read_sql_table

Define feature and target variables X and Y

In [57]:
# import libraries
import pandas as pd
import numpy as np
import os
import pickle
from sqlalchemy import create_engine
import re
import nltk

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier,AdaBoostClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag

from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, accuracy_score, f1_score, fbeta_score, classification_report

from scipy.stats import hmean
from scipy.stats.mstats import gmean
from sklearn import multioutput
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger'])

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\hp\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\hp\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\hp\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [42]:
# load data from database
cwd = os.getcwd()
dbwd = cwd.replace('/models','/data/').replace('\\models','\\data\\')
engine = create_engine('sqlite:///''DisasterResponse.db')
df = pd.read_sql_table('df',engine)
X = df['message']
Y = df.iloc[:,4:]
print(df.shape)
print(df.columns)

(26216, 41)
Index(['id', 'message', 'original', 'related', 'request', 'offer',
       'aid_related', 'medical_help', 'medical_products', 'search_and_rescue',
       'security', 'military', 'child_alone', 'water', 'food', 'shelter',
       'clothing', 'money', 'missing_people', 'refugees', 'death', 'other_aid',
       'infrastructure_related', 'transport', 'buildings', 'electricity',
       'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure',
       'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold',
       'other_weather', 'direct_report', 'direct', 'news'],
      dtype='object')


In [65]:
def load_data():
    df = pd.read_sql_table('df',engine)
    #df = df[(df["category:confidence"] == 1) & (df['category'] != 'Exclude')]
    #df.drop(['child_alone'], axis=1, inplace=True)
    print('DataFrame dim:', df.shape, '\n')
    X = df.message
    y = df.iloc[:,4:]
    return X, y

X,y=load_data()

DataFrame dim: (26216, 41) 



### 2. Write a tokenization function to process your text data

In [43]:
def tokenize(text):
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")

    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline

This machine pipeline should take in the message column as input and output classification results on the other 36 categories in the dataset. You may find the MultiOutputClassifier helpful for predicting multiple target variables.

In [44]:
def model_pipeline():
    pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier())),
    ])

    return pipeline



### 4. Train pipeline

Split data into train and test sets

Train pipeline

In [39]:
X_train, X_test, y_train, y_test = train_test_split(X, Y)

model = []
model = model_pipeline()
model.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function tokenize at...
                                                                        ccp_alpha=0.0,
                                                                        class_weight=None,
                                                                        criterion='gini',
                                                                   

Let's see is the model tells us something reasonable!

In [40]:
msg = ['Hello I see fire in the street and many houses are destroyed, homeless people everywhere']
test_output = model.predict(msg)
print(y_train.columns.values[(test_output.flatten()==1)])

['aid_related' 'direct']


### 5.Test Your Model

Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's classification_report on each.

In [45]:
y_pred = model.predict(X_test)

In [46]:
msg = ['I need your help. My house collapsed. I lost everything. My family and I are refugees.']
test_output = model.predict(msg)
print(y_train.columns.values[(test_output.flatten()==1)])

['request' 'aid_related' 'shelter' 'refugees' 'buildings' 'direct_report'
 'direct']


In [47]:
y_pred_pd = pd.DataFrame(y_pred, columns = y_test.columns)
for column in y_test.columns:
    print('------------------------------------------------------\n')
    print('FEATURE: {}\n'.format(column))
    print(classification_report(y_test[column],y_pred_pd[column]))

------------------------------------------------------

FEATURE: request

              precision    recall  f1-score   support

           0       0.90      0.99      0.94      5442
           1       0.88      0.44      0.59      1112

    accuracy                           0.89      6554
   macro avg       0.89      0.71      0.76      6554
weighted avg       0.89      0.89      0.88      6554

------------------------------------------------------

FEATURE: offer

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6528
           1       0.00      0.00      0.00        26

    accuracy                           1.00      6554
   macro avg       0.50      0.50      0.50      6554
weighted avg       0.99      1.00      0.99      6554

------------------------------------------------------

FEATURE: aid_related

              precision    recall  f1-score   support

           0       0.75      0.89      0.81      3817
           1 

C:\Users\hp\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.93      1.00      0.96      5990
           1       0.86      0.24      0.38       564

    accuracy                           0.93      6554
   macro avg       0.90      0.62      0.67      6554
weighted avg       0.93      0.93      0.91      6554

------------------------------------------------------

FEATURE: clothing

              precision    recall  f1-score   support

           0       0.98      1.00      0.99      6451
           1       0.80      0.04      0.07       103

    accuracy                           0.98      6554
   macro avg       0.89      0.52      0.53      6554
weighted avg       0.98      0.98      0.98      6554

------------------------------------------------------

FEATURE: money

              precision    recall  f1-score   support

           0       0.98      1.00      0.99      6400
           1       1.00      0.04      0.08       154

    accuracy                       

In [48]:
def multioutput_fscore(y_true,y_pred,beta=1):
    score_list = []
    if isinstance(y_pred, pd.DataFrame) == True:
        y_pred = y_pred.values
    if isinstance(y_true, pd.DataFrame) == True:
        y_true = y_true.values
    for column in range(0,y_true.shape[1]):
        score = fbeta_score(y_true[:,column],y_pred[:,column],beta,average='weighted')
        score_list.append(score)
    f1score_numpy = np.asarray(score_list)
    f1score_numpy = f1score_numpy[f1score_numpy<1]
    f1score = gmean(f1score_numpy)
    return  f1score

In [49]:
multi_f1 = multioutput_fscore(y_test,y_pred, beta = 1)
overall_accuracy = (y_pred == y_test).mean().mean()
print('Average overall accuracy {0:.2f}% \n'.format(overall_accuracy*100))
print('F1 score (custom definition) {0:.2f}%\n'.format(multi_f1*100))

Average overall accuracy 95.09% 

F1 score (custom definition) 93.49%



### 6. Improve your model

Use grid search to find better parameters.

#### ADD CUSTOM ESTIMATOR

In [50]:
class StartingVerbExtractor(BaseEstimator, TransformerMixin):

    def starting_verb(self, text):
        sentence_list = nltk.sent_tokenize(text)
        for sentence in sentence_list:
            pos_tags = nltk.pos_tag(tokenize(sentence))
            first_word, first_tag = pos_tags[0]
            if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                return True
        return False

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged)

#### Improve Model Pipeline

In [51]:
def new_model_pipeline():
    pipeline = Pipeline([
('features', FeatureUnion([

            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer())
            ])),

            ('starting_verb', StartingVerbExtractor())
        ])),

        ('clf', MultiOutputClassifier(AdaBoostClassifier()))
    ])

    return pipeline

#### DEFINE GRIDSEARCH SCORING TARGET

In [69]:
def multioutput_fscore(y_true,y_pred,beta=1):
    score_list = []
    if isinstance(y_pred, pd.DataFrame) == True:
        y_pred = y_pred.values
    if isinstance(y_true, pd.DataFrame) == True:
        y_true = y_true.values
    for column in range(0,y_true.shape[1]):
        score = fbeta_score(y_true[:,column],y_pred[:,column],beta,average='weighted')
        score_list.append(score)
    f1score_numpy = np.asarray(score_list)
    f1score_numpy = f1score_numpy[f1score_numpy<1]
    f1score = gmean(f1score_numpy)
    return  f1score

In [52]:
model.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                   dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                   lowercase=True, max_df=1.0, max_features=None, min_df=1,
                   ngram_range=(1, 1), preprocessor=None, stop_words=None,
                   strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                   tokenizer=<function tokenize at 0x0000027C112D8948>,
                   vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True,
                                                          ccp_alpha=0.0,
                                                          class_weight=None,
                                                          criterion='gini',
                                                          max_dept

#### CUSTOMIZE AND RUN GRID SEARCH

Warning It may take several hours on a standard laptop (with CPU parallelization)

In [53]:
parameters = {
    'features__text_pipeline__vect__ngram_range': ((1, 1), (1, 2)),
    'features__text_pipeline__vect__max_df': (0.75, 1.0),
  #  'features__text_pipeline__vect__max_features': (None, 5000),
  #  'features__text_pipeline__tfidf__use_idf': (True, False),
 #   'clf__n_estimators': [10, 100],
 #   'clf__learning_rate': [0.01, 0.1],
  #  'features__transformer_weights': (
  #     {'text_pipeline': 1, 'starting_verb': 0.5},
   #    {'text_pipeline': 0.5, 'starting_verb': 1},
    #   {'text_pipeline': 0.8, 'starting_verb': 1},
 #  )
}

In [54]:
scorer = make_scorer(multioutput_fscore,greater_is_better = True)

model = []
model = new_model_pipeline()
cv = GridSearchCV(model, param_grid=parameters, scoring = scorer,verbose = 2, n_jobs = -1)

cv.fit(X_train, y_train)

Fitting 5 folds for each of 4 candidates, totalling 20 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  16 out of  20 | elapsed: 27.0min remaining:  6.7min
[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed: 31.5min finished


GridSearchCV(cv=None, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('features',
                                        FeatureUnion(n_jobs=None,
                                                     transformer_list=[('text_pipeline',
                                                                        Pipeline(memory=None,
                                                                                 steps=[('vect',
                                                                                         CountVectorizer(analyzer='word',
                                                                                                         binary=False,
                                                                                                         decode_error='strict',
                                                                                                         dtype=<class 'numpy.int64'>,
               

### 7. Test your model

Show the accuracy, precision, and recall of the tuned model.

Since this project focuses on code quality, process, and pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [55]:
y_pred_pd = pd.DataFrame(y_pred, columns = y_test.columns)
for column in y_test.columns:
    print('------------------------------------------------------\n')
    print('FEATURE: {}\n'.format(column))
    print(classification_report(y_test[column],y_pred_pd[column]))

------------------------------------------------------

FEATURE: request

              precision    recall  f1-score   support

           0       0.90      0.99      0.94      5442
           1       0.88      0.44      0.59      1112

    accuracy                           0.89      6554
   macro avg       0.89      0.71      0.76      6554
weighted avg       0.89      0.89      0.88      6554

------------------------------------------------------

FEATURE: offer

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6528
           1       0.00      0.00      0.00        26

    accuracy                           1.00      6554
   macro avg       0.50      0.50      0.50      6554
weighted avg       0.99      1.00      0.99      6554

------------------------------------------------------

FEATURE: aid_related

              precision    recall  f1-score   support

           0       0.75      0.89      0.81      3817
           1 

C:\Users\hp\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))



              precision    recall  f1-score   support

           0       0.98      1.00      0.99      6437
           1       0.50      0.01      0.02       117

    accuracy                           0.98      6554
   macro avg       0.74      0.50      0.50      6554
weighted avg       0.97      0.98      0.97      6554

------------------------------------------------------

FEATURE: military

              precision    recall  f1-score   support

           0       0.97      1.00      0.98      6333
           1       0.83      0.02      0.04       221

    accuracy                           0.97      6554
   macro avg       0.90      0.51      0.51      6554
weighted avg       0.96      0.97      0.95      6554

------------------------------------------------------

FEATURE: child_alone

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6554

    accuracy                           1.00      6554
   macro avg       1.00     

              precision    recall  f1-score   support

           0       0.98      1.00      0.99      6415
           1       0.78      0.05      0.09       139

    accuracy                           0.98      6554
   macro avg       0.88      0.53      0.54      6554
weighted avg       0.98      0.98      0.97      6554

------------------------------------------------------

FEATURE: other_weather

              precision    recall  f1-score   support

           0       0.95      1.00      0.98      6237
           1       0.62      0.02      0.03       317

    accuracy                           0.95      6554
   macro avg       0.79      0.51      0.50      6554
weighted avg       0.94      0.95      0.93      6554

------------------------------------------------------

FEATURE: direct_report

              precision    recall  f1-score   support

           0       0.86      0.99      0.92      5269
           1       0.87      0.35      0.50      1285

    accuracy          

### 8. Try improving your model further. Here are a few ideas:

try other machine learning algorithms

add other features besides the TF-IDF

In [58]:
#trying to add another feature.

upd_pipeline = Pipeline([
        ('features', FeatureUnion ([
            
            ('text_pipeline', Pipeline ([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer())
                                        ])),      
             ('starting_verb', StartingVerbExtractor ())   
                                    ])),        
        
        ('clf', multioutput.MultiOutputClassifier (RandomForestClassifier ()))
        ])

# train SVM classifier
upd_pipeline.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('features',
                 FeatureUnion(n_jobs=None,
                              transformer_list=[('text_pipeline',
                                                 Pipeline(memory=None,
                                                          steps=[('vect',
                                                                  CountVectorizer(analyzer='word',
                                                                                  binary=False,
                                                                                  decode_error='strict',
                                                                                  dtype=<class 'numpy.int64'>,
                                                                                  encoding='utf-8',
                                                                                  input='content',
                                                                                  low

In [59]:
y_pred_upd = upd_pipeline.predict (X_test)
#converting to dataframe
y_pred_upd = pd.DataFrame (y_pred_upd, columns = y_test.columns)

### 9. Export your model as a pickle file

In [63]:
# save the model to disk
filename = 'classifier.pkl'
pickle.dump(model, open(filename, 'wb'))

# some time later...
 
# load the model from disk
# loaded_model = pickle.load(open(filename, 'rb'))
# result = loaded_model.score(X_test, Y_test)

### 10. Use this notebook to complete train.py

Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.

In [68]:
def main():
    X, y = load_data()
    X_train, X_test, y_train, y_test = train_test_split(X, y)

    # build pipeline
    pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier()))      
    ])     
        
    # train classifier
    pipeline.fit(X_train, y_train)
    # predict on test data
    y_pred=pipeline.predict(X_test)
    # display results
    #display_results(y_test, y_pred)
main()

DataFrame dim: (26216, 41) 

